close, volume, MA, volatility

-> 일일 수익률, RVOL, 일일 수익률의 63일 평균, 일일 수익률의 63일 표준편차

$r_t = \frac{S_t - S_{t-1}}{S_{t-1}}$

$RVOL = \frac{V}{MA_{63}(V)}$

$MA_{63}(r_t)$

$\sigma_{r_t}$

In [18]:
import pandas as pd


# 1. S&P500 티커 목록 가져오기 (예시: Wikipedia 활용)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]
sp500_tickers = sp500_df['Symbol'].tolist()


for ticker in sp500_tickers:
    try:
        df = pd.read_csv(f'./data/price/{ticker.upper()}_15yr_data.csv')
        df = df.drop([0, 1]).reset_index(drop=True)

        df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
        df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
        df['return'] = df['Close'].pct_change(1, fill_method=None)
        df['ma'] = df['return'].rolling(window=63).mean()
        df['vol'] = df['return'].rolling(window=63).std()
        df['rvol'] = df['Volume']/df['Volume'].rolling(window=63).mean()

        df.dropna(inplace=True)
        df.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)
        df.to_csv(f"./data/derived/{ticker.upper()}_der_data.csv")
        print(f"Postprocess success: {ticker.upper()}")
    except Exception as e:
        print(f'error processing {ticker}: {e}')

Postprocess success: MMM
Postprocess success: AOS
Postprocess success: ABT
Postprocess success: ABBV
Postprocess success: ACN
Postprocess success: ADBE
Postprocess success: AMD
Postprocess success: AES
Postprocess success: AFL
Postprocess success: A
Postprocess success: APD
Postprocess success: ABNB
Postprocess success: AKAM
Postprocess success: ALB
Postprocess success: ARE
Postprocess success: ALGN
Postprocess success: ALLE
Postprocess success: LNT
Postprocess success: ALL
Postprocess success: GOOGL
Postprocess success: GOOG
Postprocess success: MO
Postprocess success: AMZN
Postprocess success: AMCR
Postprocess success: AEE
Postprocess success: AEP
Postprocess success: AXP
Postprocess success: AIG
Postprocess success: AMT
Postprocess success: AWK
Postprocess success: AMP
Postprocess success: AME
Postprocess success: AMGN
Postprocess success: APH
Postprocess success: ADI
Postprocess success: ANSS
Postprocess success: AON
Postprocess success: APA
Postprocess success: APO
Postprocess suc

In [39]:
import random


def generate_scenario(n:int, k:int):
    # Number of stocks to select

    # Randomly select n stocks from the S&P 500 tickers
    selected_tickers = random.sample(sp500_tickers, n)

    # Initialize an empty DataFrame to store the concatenated data
    concatenated_df = pd.DataFrame()

    # Iterate over the selected tickers
    for i, ticker in enumerate(selected_tickers):
        try:
            # Read the derived data for the ticker
            df = pd.read_csv(f'./data/derived/{ticker.upper()}_der_data.csv')
            # Drop rows with any NaN values
            if df.isna().any().any():
                print(f'Skipping {ticker} due to NaN values')
                return 0
            
            # Rename the columns to include the stock identifier
            df = df.rename(columns={
                'return': f'S{i+1}_return',
                'ma': f'S{i+1}_ma',
                'vol': f'S{i+1}_vol',
                'rvol': f'S{i+1}_rvol'
            })
            
            # Set 'Price' as the index for joining on the same date
            df['Date'] = pd.to_datetime(df['Price'])
            df = df.set_index('Date')
            df = df.drop(columns=['Unnamed: 0', 'Price'], errors='ignore')
            # If concatenated_df is empty, initialize it with the current df
            if concatenated_df.empty:
                concatenated_df = df
            else:
                # Perform an outer join with the current df on the 'Date' index
                concatenated_df = concatenated_df.join(df, how='outer')
        
        except Exception as e:
            print(f'Error processing {ticker}: {e}')

    # Drop any columns that are duplicated due to the index or unnamed columns
    concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]

    # Filter the data to leave only consecutive one year data
    # Assuming the data is sorted by date and the 'Date' column is present
    concatenated_df = concatenated_df.reset_index()
    
    # Randomly select a year between 2008 and 2023
    random_year = random.randint(2008, 2023)
    
    # Define the start and end date for the randomly selected year
    start_date = f'{random_year}-01-01'
    end_date = f'{random_year}-12-31'

    # Filter the DataFrame for the specified one-year period
    one_year_data = concatenated_df.loc[(concatenated_df['Date'] >= start_date) & (concatenated_df['Date'] <= end_date)]
    
    # Check if the number of rows is fewer than 250
    if len(one_year_data) < 250:
        # Truncate the DataFrame
        one_year_data = pd.DataFrame()
        return 0
    # Read the macro data
    macro_df = pd.read_csv('./data/macro_15yr_data.csv')
    
    # Convert 'Date' column to datetime
    macro_df['Date'] = pd.to_datetime(macro_df['Date'])
    
    # Set 'Date' as the index for joining
    macro_df = macro_df.set_index('Date')
    
    # Perform an inner join with the macro data on the 'Date' index
    one_year_data = one_year_data.set_index('Date').join(macro_df, how='inner').reset_index()
    
    # Check for NA values
    if one_year_data.isna().any().any():
        return 0
    
    # Save the data to CSV if no NA values are present
    one_year_data.to_csv(f"./data/train_set/{k}.csv", index=False)
    return 1
    



In [ ]:
n = 10

i = 0 

while i<10000:
    i += generate_scenario(n, i)
    if i % 100 == 99:
        print(f"{i+1}th scenatio generated")



100th scenatio generated
200th scenatio generated
300th scenatio generated
400th scenatio generated
400th scenatio generated
500th scenatio generated
500th scenatio generated
500th scenatio generated
600th scenatio generated
700th scenatio generated
700th scenatio generated
800th scenatio generated
900th scenatio generated
1000th scenatio generated
1000th scenatio generated
1100th scenatio generated
1200th scenatio generated
1300th scenatio generated
1300th scenatio generated
1400th scenatio generated
1400th scenatio generated
1500th scenatio generated
1600th scenatio generated
1600th scenatio generated
1700th scenatio generated
Error processing BF.B: columns overlap but no suffix specified: Index(['Adj Close'], dtype='object')
1800th scenatio generated
1800th scenatio generated
1800th scenatio generated
1800th scenatio generated
Error processing BF.B: columns overlap but no suffix specified: Index(['Adj Close'], dtype='object')
1900th scenatio generated
1900th scenatio generated
2000t

# Data set 구축

- 10개의 주식 pool을 만든다.
    - 10개의 조합은 다양하게 구성될 수 있으므로 $_{500}C_{10}$의 경우의 수 존재, random selection
    - 1년 단위로 구성해야하기 때문에 매년 1월 1일부터 12월 31일까지의 데이터가 모두 존재하는 segment를 쪼갠다
    - 같은 연도의 segment 500개 중 10개씩을 random sampling한다.
    - 한 연도에 max 1000개정도? 로 test set을 구축한다
    - VIX 지수와 금리 데이터를 test set에 붙인다


In [8]:
import pandas as pd

snp_tickers = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'GOOGL', 'AVGO', 'TSLA', 'GOOG', 'JPM', 'LLY']
for ticker in snp_tickers:
    df = pd.read_csv(f'./data/derived/{ticker.upper()}_der_data.csv')
    print(ticker)
    print(df.iloc[0, 1])
    nan_indices = df[df.isna().any(axis=1)].index
    print("NaN이 있는 인덱스:", nan_indices.tolist())

AAPL
2008-04-03
NaN이 있는 인덱스: []
NVDA
2008-04-03
NaN이 있는 인덱스: []
MSFT
2008-04-03
NaN이 있는 인덱스: []
AMZN
2008-04-03
NaN이 있는 인덱스: []
GOOGL
2008-04-03
NaN이 있는 인덱스: []
AVGO
2009-11-04
NaN이 있는 인덱스: []
TSLA
2010-09-28
NaN이 있는 인덱스: []
GOOG
2008-04-03
NaN이 있는 인덱스: []
JPM
2008-04-03
NaN이 있는 인덱스: []
LLY
2008-04-03
NaN이 있는 인덱스: []
